<p>Markov switching autoregression model in <code>statespace.regime_switching</code> module has the following specification:</p>
<p>$$ \phi_{S_t}(L)(y_t - \mu_{S_t} - x_t' \beta_t) = e_t \\
        e_t \sim N(0, \sigma_{S_t}^2) $$</p>
<p>Where $\phi_{S_t}$ is a regime switching lag polynomial, $y_t$ is an observed process, $\mu_{S_t}$ is a switching intercept, $x_t$ is exogenous vector , $\beta_t$ are exogenous regressors and $e_t$ is a white noise. Switching objects in the specification can optionally be used without switching.</p>
<p>Presented code uses this model for business fluctuations modelling (chapter 4.4 of Kim and Nelson, 1999). Only intercept terms are switching in this case.</p>
<p>See also http://econ.korea.ac.kr/~cjkim/MARKOV/programs/hmt4_kim.opt - the same model implementation used for testing.</p>

In [ ]:
"""
Hamilton's (1989) Markov-Switching Model of Business Fluctuations (chapter 4.4 of Kim and Nelson, 1999)

Author: Valery Likhosherstov
License: Simplified-BSD
"""

import numpy as np
from statsmodels.tsa.statespace.regime_switching.api import MarkovAutoregression
from statsmodels.tsa.statespace.regime_switching.tests.results import results_hamilton1989

dtype = np.float64

# Model attributes

# The number of Markov switching autoregression regimes. Note, that this
# is different from the number of regimes, used internally in state-space
# model.
k_ar_regimes = 2

# Autoregression order
order = 4

true = results_hamilton1989.hmt4_kim

# Preparing observations
data = np.array(true['data'], dtype=dtype)
data = np.log(data) * 100

obs = data[20:152] - data[19:151]

# Instantiate MS-AR model with switching mean
model = MarkovAutoregression(k_ar_regimes, order, obs, switching_mean=True, dtype=dtype)

In [ ]:
# Fit the model from start parameters, used by authors.
# Note, that it takes some time to converge.
results = model.fit(start_params=np.array(true['untransformed_start_parameters'], dtype=dtype),
        transformed=False)

# Print the summary
print(results.summary())

In [ ]:
# EM-algorithm feature for Markov autoregression model.


# This method, by default, runs 50 sessions of EM-algorithm from random starts with 5 iterations each.
# (It takes some time to run this code)
params = model.fit_em_with_random_starts()

# Get the results object
em_results = model.smooth(params)

# Print the summary
print(em_results.summary())

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

# Example of prediction and forecasting

# observation period
years = np.arange(1947.25, 1996, 0.25)[20:152]

# Forecast until
forecast_end = 2000

# Range of years for forecast
forecast_years = np.arange(years[-1] + 0.25, forecast_end, 0.25)

# Perform prediction and forecasting
predict = results.get_prediction()
forecast = results.get_forecast(steps=forecast_years.shape[0])

# Plot prediction and forecast

fig, ax = plt.subplots(figsize=(10,4))

ax.plot(years, obs, 'k.', label='Observations')

# Cut `order` prefix from `years` range, since in Markov autoregression uses
# first `order` observations for initialization
ax.plot(years[order:], predict.predicted_mean, label='One-step-ahead Prediction')
predict_ci = predict.conf_int(alpha=0.05)
ax.fill_between(years[order:], predict_ci.iloc[:, 0], predict_ci.iloc[:, 1], alpha=0.1)

ax.plot(forecast_years, forecast.predicted_mean, 'r', label='Forecast')
forecast_ci = forecast.conf_int()
ax.fill_between(forecast_years, forecast_ci.iloc[:, 0], forecast_ci.iloc[:, 1], alpha=0.1)

ax.set_ylim((-14, 8))
legend = ax.legend(loc='lower left');

<h1>References</h1>
<p>Kim, Chang-Jin, and Charles R. Nelson. 1999.<br>
"State-Space Models with Regime Switching: Classical and Gibbs-Sampling Approaches with Applications".<br>
MIT Press Books. The MIT Press.</p>